In [ ]:
#default_exp models

In [ ]:
#hide
from nbdev import *
%load_ext autoreload
%autoreload 2

# Models

> Uniserie models implementations.

In [ ]:
#hide
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
from nbdev.showdoc import show_doc

In [ ]:
#export
from itertools import count
from numbers import Number
from typing import Collection, List, Optional, Sequence, Tuple

import numpy as np
import pandas as pd
from numba import njit
from scipy.optimize import minimize

from statsforecast.arima import auto_arima_f, forecast_arima, fitted_arima
from statsforecast.ets import ets_f, forecast_ets

## ARIMA methods 

In [ ]:
#export
def auto_arima(X: np.ndarray, # time series
               h: int, # forecasting horizon
               future_xreg=None, #future regressors 
               fitted: bool=False, # fitted values
               season_length: int=1, # season_length
               approximation: bool=False, # approximation 
               level: Optional[Tuple[int]] = None # level
    ) -> np.ndarray: 
    y = X[:, 0] if X.ndim == 2 else X
    xreg = X[:, 1:] if (X.ndim == 2 and X.shape[1] > 1) else None
    with np.errstate(invalid='ignore'):
        mod = auto_arima_f(
            y, 
            xreg=xreg,
            period=season_length, 
            approximation=approximation,
            allowmean=False, allowdrift=False #not implemented yet
        )
    fcst = forecast_arima(mod, h, xreg=future_xreg, level=level)
    mean = fcst['mean']
    if fitted:
        return {'mean': mean, 'fitted': fitted_arima(mod)}
    if level is None:
        return {'mean': mean}
    return {
        'mean': mean,
        **{f'lo-{l}': fcst['lower'][f'{l}%'] for l in reversed(level)},
        **{f'hi-{l}': fcst['upper'][f'{l}%'] for l in level},
    } 

In [ ]:
show_doc(auto_arima)

**Auto ARIMA**: Automatically selects the best ARIMA (AutoRegressive Integrated Moving Average) model using an information criterion. Default is the corrected Akaike Information Criterion (AICc). Information criterions are tests used to check how well a model fits the data it is trying to describe. 

The ARIMA models are based in the autocorrelations in the data and the autocorrelations of the forecast errors. Every model has three components: AR, I, and MA. 

An AR(p) model captures the autocorrelations in the data at lags $1,2,\dots p$. 

$$y_t = \beta_0+\beta_1y_{t-1}+\beta_2y_{t-2}+\dots+\beta_py{t_p}+\epsilon_t$$

If the autocorrelations of the forecast errors are added up to lag $q$, an ARMA(p,q) model is obtained. 

$$y_t = \beta_0+\beta_1y_{t-1}+\beta_2y_{t-2}+\dots+\beta_py{t_p} + \epsilon_t+\theta_1 \epsilon_{t-1}+\theta_2\epsilon_{t-2}+\dots\theta_q\epsilon_{t-q}$$

The last component of an ARIMA model is the integrated (I) part, which is a differencing operation. The order of differencing, denoted by $d$, indicates how many rounds of lag-1 differencing are performed. 

ARIMA models requiere the user to select $p$, $q$, and $d$. The values for $\beta_i, i=1,2,\dots,p$ and $\theta_j, j=1,2,\dots,q$ are then estimated. Auto ARIMA automatically makes this selection, searching over a range of possible values for $p$, $q$, and $d$, and then choosing the best model using an information criterion. 

## Exponential smoothing methods 

In [ ]:
#export
def ets(X: np.ndarray, # time series
        h: int, # forecasting horizon
        future_xreg=None, # future regressors
        fitted: bool = False, # fitted values
        season_length: int = 1, # season length
        model: str = 'ZZZ' # model type
    )-> np.ndarray: 
    y = X[:, 0] if X.ndim == 2 else X
    xreg = X[:, 1:] if (X.ndim == 2 and X.shape[1] > 1) else None
    mod = ets_f(y, m=season_length, model=model)
    fcst = forecast_ets(mod, h)
    keys = ['mean']
    if fitted:
        keys.append('fitted')
    return {key: fcst[key] for key in keys}

In [ ]:
show_doc(ets)

**Error, Trend, and Seasonality**: Statistical models for exponential smoothing. These models are stochastic data generating processes than can produce complete forecast distributions. Each model consists of a set of equations to describe the observed data and the unobserved components or states, which are level, trend, and seasonal. Errors can be either additive or multiplicative. The notation ETS(Z,Z,Z) is used to describe the ETS model being used, where Z can take one of the following values. 

Notation| 
----- |
N = None |     
A = Additive |   
Ad = Additive (damped) | 
M = Multiplicative | 

The possibilities for each state are shown below. 

State | Possible values 
----- | -----
Error | A, M
Trend | N, A, Ad
Seasonal | N, A, M

In [ ]:
#export
@njit
def _ses_fcst_mse(x: np.ndarray, alpha: float) -> Tuple[float, float]:
    """Perform simple exponential smoothing on a series.

    This function returns the one step ahead prediction
    as well as the mean squared error of the fit.
    """
    smoothed = x[0]
    n = x.size
    mse = 0.
    fitted = np.full(n, np.nan, np.float32)

    for i in range(1, n):
        smoothed = (alpha * x[i - 1] + (1 - alpha) * smoothed).item()
        error = x[i] - smoothed
        mse += error * error
        fitted[i] = smoothed

    mse /= n
    forecast = alpha * x[-1] + (1 - alpha) * smoothed
    return forecast, mse, fitted


def _ses_mse(alpha: float, x: np.ndarray) -> float:
    """Compute the mean squared error of a simple exponential smoothing fit."""
    _, mse, _ = _ses_fcst_mse(x, alpha)
    return mse


@njit
def _ses_forecast(x: np.ndarray, alpha: float) -> float:
    """One step ahead forecast with simple exponential smoothing."""
    forecast, _, fitted = _ses_fcst_mse(x, alpha)
    return forecast, fitted


@njit
def _demand(x: np.ndarray) -> np.ndarray:
    """Extract the positive elements of a vector."""
    return x[x > 0]


@njit
def _intervals(x: np.ndarray) -> np.ndarray:
    """Compute the intervals between non zero elements of a vector."""
    y = []

    ctr = 1
    for val in x:
        if val == 0:
            ctr += 1
        else:
            y.append(ctr)
            ctr = 1

    y = np.array(y)
    return y


@njit
def _probability(x: np.ndarray) -> np.ndarray:
    """Compute the element probabilities of being non zero."""
    return (x != 0).astype(np.int32)


def _optimized_ses_forecast(
        x: np.ndarray,
        bounds: Sequence[Tuple[float, float]] = [(0.1, 0.3)]
    ) -> float:
    """Searches for the optimal alpha and computes SES one step forecast."""
    alpha = minimize(
        fun=_ses_mse,
        x0=(0,),
        args=(x,),
        bounds=bounds,
        method='L-BFGS-B'
    ).x[0]
    forecast, fitted = _ses_forecast(x, alpha)
    return forecast, fitted


@njit
def _chunk_sums(array: np.ndarray, chunk_size: int) -> np.ndarray:
    """Splits an array into chunks and returns the sum of each chunk."""
    n = array.size
    n_chunks = n // chunk_size
    sums = np.empty(n_chunks)
    for i, start in enumerate(range(0, n, chunk_size)):
        sums[i] = array[start : start + chunk_size].sum()
    return sums

In [ ]:
#exporti
#@njit
def ses_(X: np.ndarray, # time series
        h: int, # forecasting horizon 
        future_xreg: np.ndarray, # future regressors
        fitted: bool, # fitted values
        alpha: float): # smoothing parameter
    y = X[:, 0] if X.ndim == 2 else X
    fcst, _, fitted_vals = _ses_fcst_mse(y, alpha)
    mean = np.full(h, fcst, np.float32)
    fcst = {'mean': mean}
    if fitted:
        fcst['fitted'] = fitted_vals
    return fcst

In [ ]:
#export
def ses(X: np.ndarray, # time series
        h: int, # forecasting horizon 
        future_xreg: np.ndarray, # future regressors
        fitted: bool, # fitted values
        alpha: float): # smoothing parameter
    
    return ses_(X, h, future_xreg, fitted, alpha)

In [ ]:
#hide
from statsforecast.utils import AirPassengers as ap

In [ ]:
#hide
fcst_ses = ses(ap, 12, None, True, 0.1)
test_close(fcst_ses['mean'], np.repeat(460.3028, 12), eps=1e-4)
#to recover these residuals from R
#you have to pass initial="simple"
#in the `ses` function
np.testing.assert_allclose(fcst_ses['fitted'][[0, 1, -1]], np.array([np.nan, 118 - 6., 432 + 31.447525]))

In [ ]:
show_doc(ses, name = 'ses')

**Simple (or single) exponential smoothing**: Uses a weighted average of all past observations where the weights decrease exponentially into the past. Suitable for data with no clear trend or seasonality. Assuming there are $t$ observations, the one-step forecast is given by

$$\hat{y}_{t+1}= \alpha y_{t} + \alpha(1-\alpha)y_{t-1} + \alpha(1-\alpha)^2 y_{t-2} \dots$$

which can also be written as 

$$\hat{y}_{t+1} = \alpha y_t + \alpha(1-\alpha) \hat{y}_{t-1}$$

The rate $0 \leq \alpha \leq 1$ at which the weights decrease is called the smoothing parameter. When $\alpha = 1$, SES is equal to the naive method. 

In [ ]:
#export
def ses_optimized(X: np.ndarray, # time series 
                  h: int, # forecasting horizon
                  future_xreg: np.ndarray, # future regressors
                  fitted: bool): # fitted values
    y = X[:, 0] if X.ndim == 2 else X
    fcst, res = _optimized_ses_forecast(y, [(0.01, 0.99)])
    mean = np.full(h, fcst, np.float32)
    fcst = {'mean': mean}
    if fitted:
        fcst['fitted'] = fitted
    return fcst

In [ ]:
#hide
fcst_ses_optimized = ses_optimized(ap, 12, None, True)

In [ ]:
show_doc(ses_optimized)

**Simple exponential smoothing optimized**: A version of SES where the smoothing parameter $\alpha$ is chosen automatically by minimizing the mean squared error of the fit. 

In [ ]:
#exporti
@njit
def seasonal_exponential_smoothing_(X: np.ndarray, # time series
                                   h: int, # forecasting horizon 
                                   future_xreg: np.ndarray, # future regressors 
                                   fitted: bool, # fitted values 
                                   season_length: int, # length of season
                                   alpha: float): # smoothing parameter
    y = X[:, 0] if X.ndim == 2 else X
    if y.size < season_length:
        return {'mean': np.full(h, np.nan, np.float32)}
    season_vals = np.empty(season_length, np.float32)
    fitted_vals = np.full(y.size, np.nan, np.float32)
    for i in range(season_length):
        season_vals[i], fitted_vals[i::season_length] = _ses_forecast(y[i::season_length], alpha)
    out = np.empty(h, np.float32)
    for i in range(h):
        out[i] = season_vals[i % season_length]
    fcst = {'mean': out}
    if fitted:
        fcst['fitted'] = fitted_vals
    return fcst

In [ ]:
#export
#@njit
def seasonal_exponential_smoothing(X: np.ndarray, # time series
                                   h: int, # forecasting horizon 
                                   future_xreg: np.ndarray, # future regressors 
                                   fitted: bool, # fitted values 
                                   season_length: int, # length of season
                                   alpha: float): # smoothing parameter
    
    return seasonal_exponential_smoothing_(X,h,future_xreg,fitted,season_length,alpha)

In [ ]:
#hide
# `seasonal_exponential_smoothing`
# should recover seasonal_naive when alpha=1.
fcst_seas_ses = seasonal_exponential_smoothing(ap, 12, None, True, 12, 1.)
test_eq(fcst_seas_ses['fitted'][-3:],  np.array([461 - 54., 390 - 28., 432 - 27.]))
#np.testing.assert_array_equal(
#    fcst_seas_ses['fitted'], 
#    fcst_seas_naive['fitted']
#)

In [ ]:
show_doc(seasonal_exponential_smoothing)

**Seasonal exponential smoothing**: A seasonal version of exponential smoothing. 

In [ ]:
#export
def seasonal_ses_optimized(X: np.ndarray, # time series 
                           h: int, # forecasting horizon
                           future_xreg: np.ndarray, # future regressors  
                           fitted: bool , # fitted values 
                           season_length: int): # season length
    y = X[:, 0] if X.ndim == 2 else X
    if y.size < season_length:
        return {'mean': np.full(h, np.nan, np.float32)}
    season_vals = np.empty(season_length, np.float32)
    fitted_vals = np.full(y.size, np.nan, np.float32)
    for i in range(season_length):
        season_vals[i], fitted_vals[i::season_length] = _optimized_ses_forecast(y[i::season_length], [(0.01, 0.99)])
    out = np.empty(h, np.float32)
    for i in range(h):
        out[i] = season_vals[i % season_length]
    fcst = {'mean': out}
    if fitted:
        fcst['fitted'] = fitted_vals
    return fcst

In [ ]:
#hide
fcst_seas_seas_opt = seasonal_ses_optimized(ap, 12, None, True, 12)

In [ ]:
show_doc(seasonal_ses_optimized)

**Seasonal SES optimized**: A seasonal version of SES optimized 

## Simple methods

In [ ]:
#exporti
@njit
def historic_average_(X: np.ndarray, # time series 
                     h: int, # forecasting horizon
                     future_xreg: np.ndarray, # future regressors
                     fitted: bool): # fitted values
    y = X[:, 0] if X.ndim == 2 else X
    mean = np.repeat(y.mean(), h)
    fcst = {'mean': mean}
    if fitted:
        fitted_vals = np.full(y.size, np.nan, y.dtype)
        fitted_vals[1:] = y.cumsum()[:-1] / np.arange(1, y.size)
        fcst['fitted'] = fitted_vals
    return fcst

In [ ]:
#export
#@njit
def historic_average(X: np.ndarray, # time series 
                     h: int, # forecasting horizon
                     future_xreg: np.ndarray, # future regressors
                     fitted: bool): # fitted values
    
    return historic_average_(X, h, future_xreg, fitted)

In [ ]:
#hide
from statsforecast.utils import AirPassengers as ap

In [ ]:
#hide
fcst_ha = historic_average(ap, 12, None, True)
test_eq(fcst_ha['mean'], np.repeat(ap.mean(), 12))
np.testing.assert_almost_equal(
    fcst_ha['fitted'][:4],
    np.array([np.nan, 112., 115., 120.6666667])
)

In [ ]:
show_doc(historic_average)

**Historic average:** Also known as mean method. Uses a simple average of all past observations. Assuming there are $t$ observations, the one-step forecast is given by 

$$ \hat{y}_{t+1} = \frac{1}{t} \sum_{j=1}^t y_j $$


In [ ]:
#exporti
@njit
def naive_(X: np.ndarray, # time series
          h: int, # forecasting horizon
          future_xreg: np.ndarray, # future regressors 
          fitted: bool): # fitted values 
    y = X[:, 0] if X.ndim == 2 else X
    mean = np.repeat(y[-1], h).astype(np.float32)
    if fitted:
        fitted_vals = np.full(y.size, np.nan, np.float32)
        fitted_vals[1:] = np.roll(y, 1)[1:]
        return {'mean': mean, 'fitted': fitted_vals}
    return {'mean': mean}

In [ ]:
#export
#@njit
def naive(X: np.ndarray, # time series
          h: int, # forecasting horizon
          future_xreg: np.ndarray, # future regressors 
          fitted: bool): # fitted values 
    
    return naive_(X, h, future_xreg, fitted)

In [ ]:
show_doc(naive)

**Naive**: A random walk model, defined as 

$$ \hat{y}_{t+1} = y_t $$

In [ ]:
#exporti
@njit
def random_walk_with_drift_(X: np.ndarray, # time series
                           h: int, # forecasting horizon
                           future_xreg: np.ndarray, # future regressors
                           fitted: bool): # fitted values
    y = X[:, 0] if X.ndim == 2 else X
    slope = (y[-1] - y[0]) / (y.size - 1)
    mean = slope * (1 + np.arange(h)) + y[-1]
    fcst = {'mean': mean.astype(np.float32)}
    if fitted:
        fitted_vals = np.full(y.size, np.nan, dtype=np.float32)
        fitted_vals[1:] = (slope + y[:-1]).astype(np.float32)
        fcst['fitted'] = fitted_vals
    return fcst

In [ ]:
#export
#@njit
def random_walk_with_drift(X: np.ndarray, # time series
                           h: int, # forecasting horizon
                           future_xreg: np.ndarray, # future regressors
                           fitted: bool): # fitted values
    
    return random_walk_with_drift_(X, h, future_xreg, fitted)

In [ ]:
#hide
fcst_rwd = random_walk_with_drift(ap, 12, None, True)
test_close(fcst_rwd['mean'][:2], np.array([434.2378, 436.4755]), eps=1e-4)
np.testing.assert_almost_equal(
    fcst_rwd['fitted'][:3], 
    np.array([np.nan, 118 - 3.7622378, 132 - 11.7622378]),
    decimal=6
)

In [ ]:
show_doc(random_walk_with_drift)

**Random walk with drift**: A variation of the naive method allows the forecasts to change over time. The amout of change, called drift, is the average change seen in the historical data. 

$$ \hat{y}_{t+1} = y_t+\frac{1}{t-1}\sum_{j=1}^t (y_j-y_{j-1}) = y_t+ \frac{y_t-y_1}{t-1} $$

From the previous equation, we can see that this is equivalent to extrapolating a line between the first and the last observation. 

In [ ]:
#exporti
@njit
def seasonal_naive_(X: np.ndarray, # time series
                   h: int, # forecasting horizon
                   future_xreg: np.ndarray, #future regressors
                   fitted: bool, #fitted values 
                   season_length: int): # season length
    y = X[:, 0] if X.ndim == 2 else X
    if y.size < season_length:
        return {'mean': np.full(h, np.nan, np.float32)}
    season_vals = np.empty(season_length, np.float32)
    fitted_vals = np.full(y.size, np.nan, np.float32)
    for i in range(season_length):
        s_naive = naive_(y[i::season_length], 1, None, fitted)
        season_vals[i] = s_naive['mean'].item()
        if fitted:
            fitted_vals[i::season_length] = s_naive['fitted']
    out = np.empty(h, np.float32)
    for i in range(h):
        out[i] = season_vals[i % season_length]
    fcst = {'mean': out}
    if fitted:
        fcst['fitted'] = fitted_vals
    return fcst

In [ ]:
#export
#@njit
def seasonal_naive(X: np.ndarray, # time series
                   h: int, # forecasting horizon
                   future_xreg: np.ndarray, #future regressors
                   fitted: bool, #fitted values 
                   season_length: int): # length of season
    
    return seasonal_naive_(X, h, future_xreg, fitted, season_length)

In [ ]:
#hide
fcst_seas_naive = seasonal_naive(ap, 12, None, True, 12)
test_eq(fcst_seas_naive['fitted'][-3:], np.array([461 - 54., 390 - 28., 432 - 27.]))

In [ ]:
show_doc(seasonal_naive)

**Seasonal naive**: Similar to the naive method, but uses the last known observation of the same period (e.g. the same month of the previous year) in order to capture seasonal variations. 

In [ ]:
#exporti
@njit
def window_average_(X: np.ndarray, # time series 
                   h: int, # forecasting horizon 
                   future_xreg: np.ndarray, # future regressors
                   fitted: bool, # fitted values
                   window_size: int): # window size
    if fitted:
        raise NotImplementedError('return fitted')
    y = X[:, 0] if X.ndim == 2 else X
    if y.size < window_size:
        return {'mean': np.full(h, np.nan, np.float32)}
    wavg = y[-window_size:].mean()
    mean = np.repeat(wavg, h)
    return {'mean': mean}

In [ ]:
#export
#@njit
def window_average(X: np.ndarray, # time series 
                   h: int, # forecasting horizon 
                   future_xreg: np.ndarray, # future regressors
                   fitted: bool, # fitted values
                   window_size: int): # window size
    
    return window_average_(X, h, future_xreg, fitted, window_size)

In [ ]:
show_doc(window_average)

**Window average**: Uses the average of the last $k$ observations, with $k$ the length of the window. Wider windows will capture global trends, while narrow windows will reveal local trends. The length of the window selected should take into account the importance of past observations and how fast the series changes. 

In [ ]:
#exporti
@njit
def seasonal_window_average_(
    X: np.ndarray,
    h: int,
    future_xreg: np.ndarray,
    fitted: bool,
    season_length: int,
    window_size: int,
) -> np.ndarray:
    if fitted:
        raise NotImplementedError('return fitted')
    y = X[:, 0] if X.ndim == 2 else X
    min_samples = season_length * window_size
    if y.size < min_samples:
        return {'mean': np.full(h, np.nan, np.float32)}
    season_avgs = np.zeros(season_length, np.float32)
    for i, value in enumerate(y[-min_samples:]):
        season = i % season_length
        season_avgs[season] += value / window_size
    out = np.empty(h, np.float32)
    for i in range(h):
        out[i] = season_avgs[i % season_length]
    return {'mean': out}

In [ ]:
#export
#@njit
def seasonal_window_average(
    X: np.ndarray, # time series
    h: int, # forecasting horizon
    future_xreg: np.ndarray, # future regressors
    fitted: bool, # fitted values
    season_length: int, # length of season
    window_size: int, # window size
) -> np.ndarray:
    
    return seasonal_window_average_(X,h,future_xreg, fitted, season_length, window_size)

In [ ]:
show_doc(seasonal_window_average)

**Seasonal window average**: An average of the last $k$ observations of the same period, with $k$ the length of the window.

## Sparse or intermittent series 

Sparse or intermittent series are series with very few non-zero observations. They are notoriously hard to forecast, and so, different methods have been developed especifically for them. Before the development of Croston's method and its variants, SES was usually used to forecast them. 

In [ ]:
#export
def adida(X: np.ndarray, # time series
          h: int, # forecasting horizon
          future_xreg: np.ndarray, # future regressors 
          fitted: bool): # fitted values
    if fitted:
        raise NotImplementedError('return fitted')
    y = X[:, 0] if X.ndim == 2 else X
    if (y == 0).all():
        return {'mean': np.repeat(np.float32(0), h)}
    y_intervals = _intervals(y)
    mean_interval = y_intervals.mean()
    aggregation_level = round(mean_interval)
    lost_remainder_data = len(y) % aggregation_level
    y_cut = y[lost_remainder_data:]
    aggregation_sums = _chunk_sums(y_cut, aggregation_level)
    sums_forecast, _ = _optimized_ses_forecast(aggregation_sums)
    forecast = sums_forecast / aggregation_level
    mean = np.repeat(forecast, h)
    return {'mean': mean}

In [ ]:
show_doc(adida)

**Aggregate-Dissagregate Intermittent Demand Approach**: Uses temporal aggregation to reduce the number of zero observations. Once the data has been agregated, it uses the optimized SES to generate the forecasts at the new level. It then breaks down the forecast to the original level using equal weights.  

In [ ]:
#exporti
@njit
def croston_classic_(X: np.ndarray, # time series
                    h: int, # forecasting horizon  
                    future_xreg: np.ndarray, # future regressors
                    fitted: bool): # fitted values
    if fitted:
        raise NotImplementedError('return fitted')
    y = X[:, 0] if X.ndim == 2 else X
    yd = _demand(y)
    yi = _intervals(y)
    ydp, _ = _ses_forecast(yd, 0.1)
    yip, _ = _ses_forecast(yi, 0.1)
    mean = ydp / yip
    return {'mean': mean}

In [ ]:
#export
#@njit
def croston_classic(X: np.ndarray, # time series
                    h: int, # forecasting horizon  
                    future_xreg: np.ndarray, # future regressors
                    fitted: bool): # fitted values
    
    return croston_classic_(X, h, future_xreg, fitted)

In [ ]:
show_doc(croston_classic)

**Croston classic**: A method to forecast time series that exhibit intermittent demand. It decomposes the original time series into a non-zero demand size $z_t$ and inter-demand intervals $p_t$. Then the forecast is given by 

$$ \hat{y}_t = \frac{\hat{z}_t}{\hat{p}_t} $$ 

where $\hat{z}_t$ and $\hat{p}_t$ are forecasted using SES. The smoothing parameter of both components is set equal to 0.1

In [ ]:
#export
def croston_optimized(X: np.ndarray, # time series
                      h: int, # forecasting horizon
                      future_xreg: np.ndarray, # future regressors
                      fitted: bool): # fitted values
    if fitted:
        raise NotImplementedError('return fitted')
    y = X[:, 0] if X.ndim == 2 else X
    yd = _demand(y)
    yi = _intervals(y)
    ydp, _ = _optimized_ses_forecast(yd)
    yip, _ = _optimized_ses_forecast(yi)
    mean = ydp / yip
    return {'mean': mean}

In [ ]:
show_doc(croston_optimized)

**Croston Optimized**: A variation of the classic Croston's method where the smooting paramater is optimally selected from the range $[0.1,0.3]$. Both the non-zero demand $z_t$ and the inter-demand intervals $p_t$ are smoothed separately, so their smoothing parameters can be different. 

In [ ]:
#exporti
@njit
def croston_sba_(X: np.ndarray, # time series
                h: int, # forecasting horizon
                future_xreg: np.ndarray, # future regressors
                fitted: bool): # fitted values
    if fitted:
        raise NotImplementedError('return fitted')
    y = X[:, 0] if X.ndim == 2 else X
    mean = croston_classic_(y, h, future_xreg, fitted)
    mean['mean'] *= 0.95
    return mean

In [ ]:
#export
#@njit
def croston_sba(X: np.ndarray, # time series
                h: int, # forecasting horizon
                future_xreg: np.ndarray, # future regressors
                fitted: bool): # fitted values
    
    return croston_sba_(X, h, future_xreg, fitted)


In [ ]:
show_doc(croston_sba)

**Croston with Syntetos-Boylan Approximation**: A variation of the classic Croston's method that uses a debiasing factor, so that the forecast is given by 

$$ \hat{y}_t = 0.95  \frac{\hat{z}_t}{\hat{p}_t} $$ 


In [ ]:
#export
def imapa(X: np.ndarray, # time series
          h: int, # forecasting horizon
          future_xreg: np.ndarray, # future regressors 
          fitted: bool): # fitted values
    if fitted:
        raise NotImplementedError('return fitted')
    y = X[:, 0] if X.ndim == 2 else X
    if (y == 0).all():
        return {'mean': np.repeat(np.float32(0), h)}
    y_intervals = _intervals(y)
    mean_interval = y_intervals.mean().item()
    max_aggregation_level = round(mean_interval)
    forecasts = np.empty(max_aggregation_level, np.float32)
    for aggregation_level in range(1, max_aggregation_level + 1):
        lost_remainder_data = len(y) % aggregation_level
        y_cut = y[lost_remainder_data:]
        aggregation_sums = _chunk_sums(y_cut, aggregation_level)
        forecast, _ = _optimized_ses_forecast(aggregation_sums)
        forecasts[aggregation_level - 1] = (forecast / aggregation_level)
    forecast = forecasts.mean()
    mean = np.repeat(forecast, h)
    return {'mean': mean}

In [ ]:
show_doc(imapa)

**Intermittent Multiple Aggregation Prediction Algorithm**: Similar to ADIDA, but instead of using a single aggregation level, it considers multiple in order to capture different dynamics of the data. Uses the optimized SES to generate the forecasts at the new levels and then combines them using a simple average. 

In [ ]:
#exporti
@njit
def tsb_(X: np.ndarray, # time series
        h: int, # forecasting horizon 
        future_xreg: np.ndarray, # future regressors
        fitted: int, # fitted values 
        alpha_d: float,
        alpha_p: float):
    if fitted:
        raise NotImplementedError('return fitted')
    y = X[:, 0] if X.ndim == 2 else X
    if (y == 0).all():
        return {'mean': np.repeat(np.float32(0), h)}
    yd = _demand(y)
    yp = _probability(y)
    ypf, _ = _ses_forecast(yp, alpha_p)
    ydf, _ = _ses_forecast(yd, alpha_d)
    forecast = np.float32(ypf * ydf)
    mean = np.repeat(forecast, h)
    return {'mean': mean}

In [ ]:
#export
#@njit
def tsb(X: np.ndarray, # time series
        h: int, # forecasting horizon 
        future_xreg: np.ndarray, # future regressors
        fitted: int, # fitted values 
        alpha_d: float,
        alpha_p: float):
    
    return tsb_(X, h, future_xreg, fitted, alpha_d, alpha_p)

In [ ]:
show_doc(tsb)

**Teunter-Syntetos-Babai**: A modification of Croston's method that replaces the inter-demand intervals with the demand probability $d_t$, which is defined as follows. 

$$
d_t = \begin{cases}
    1  & \text{if demand occurs at time t}\\ 
    0 & \text{otherwise.}
\end{cases}
$$

Hence, the forecast is given by 

$$\hat{y}_t= \hat{d}_t\hat{z_t}$$

Both $d_t$ and $z_t$ are forecasted using SES. The smooting paramaters of each may differ, like in the optimized Croston's method.

## References

#### **General**
- Hyndman, R.J., & Athanasopoulos, G. (2021) Forecasting: principles and practice, 3rd edition, OTexts: Melbourne, Australia. [OTexts.com/fpp3](https://otexts.com/fpp3/). Accessed on July 2022.

- Shmueli, G., & Lichtendahl Jr, K. C. (2016). Practical time series forecasting with r: A hands-on guide. Axelrod Schnall Publishers.

#### **For sparse or intermittent series**

- [Croston, J. D. (1972). Forecasting and stock control for intermittent demands. Journal of the Operational Research Society, 23(3), 289-303.](https://link.springer.com/article/10.1057/jors.1972.50)


- [Nikolopoulos, K., Syntetos, A. A., Boylan, J. E., Petropoulos, F., & Assimakopoulos, V. (2011). An aggregate–disaggregate intermittent demand approach (ADIDA) to forecasting: an empirical proposition and analysis. Journal of the Operational Research Society, 62(3), 544-554.](https://researchportal.bath.ac.uk/en/publications/an-aggregate-disaggregate-intermittent-demand-approach-adida-to-f)

- [Syntetos, A. A., & Boylan, J. E. (2005). The accuracy of intermittent demand estimates. International Journal of forecasting, 21(2), 303-314.](https://www.academia.edu/1527250/The_accuracy_of_intermittent_demand_estimates)

- Syntetos, A. A., & Boylan, J. E. (2021). Intermittent demand forecasting: Context, methods and applications. John Wiley & Sons.

- [Teunter, R. H., Syntetos, A. A., & Babai, M. Z. (2011). Intermittent demand: Linking forecasting to inventory obsolescence. European Journal of Operational Research, 214(3), 606-615.](https://www.sciencedirect.com/science/article/abs/pii/S0377221711004437)


In [ ]:
from statsforecast.utils import AirPassengers as ap

In [ ]:
auto_arima(ap, 12, season_length=12)

In [ ]:
ets(ap, 12, season_length=12, fitted=True)

External regressors

In [ ]:
drift = np.arange(1, ap.size + 1)
X = np.vstack([ap, np.log(drift), np.sqrt(drift)]).T

In [ ]:
newdrift = np.arange(ap.size + 1, ap.size + 7 + 1).reshape(-1, 1)
newxreg = np.concatenate([np.log(newdrift), np.sqrt(newdrift)], axis=1)

In [ ]:
auto_arima(X, 7, future_xreg=newxreg, season_length=12)

Confidence intervals

In [ ]:
pd.DataFrame(auto_arima(ap, 12, season_length=12, level=(80, 95)))